In [1]:
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-cfd6  GCE       5                                       RUNNING  us-central1-a


In [2]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [ ]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage
import math
import numpy as np
import builtins
from contextlib import closing
import hashlib
from time import time #we use it for testing

def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

In [ ]:
!ls -l /usr/lib/spark/jars/graph*

In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [ ]:
bucket_name = '3b1u8c3k6e4t122' 
client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    print(b.name)

In [ ]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

In [ ]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [ ]:
from inverted_index_gcp import InvertedIndex
from inverted_index_gcp import MultiFileReader
from inverted_index_gcp import MultiFileWriter


In [ ]:
#rdds of which we build the different indexes on
body_data = parquetFile.select("id","text").rdd
title_data = parquetFile.select("id","title").rdd
anchor_data = parquetFile.select("id","anchor_text").rdd
anchor_data= anchor_data.flatMap(lambda l: [(value[0], value[1]) for value in l[1]])  
anchor_data= anchor_data.groupByKey().mapValues(list)
anchor_data=anchor_data.mapValues(lambda x: ','.join(set(item for item in x)))

In [ ]:
#using code from assignment 3- gcp
#funcs to help build the inverted index
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became", "redirect","user"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def calc_doc_len(text, id):
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  tokens = [token for token in tokens if token not in all_stopwords]
  
  return len(tokens)

def word_count(text, id):
  
  ''' Count the frequency of each word in `text` (tf) that is not included in 
  `all_stopwords` and return entries that will go into our posting lists. 
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs 
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  tokens = [token for token in tokens if token not in all_stopwords ]
  word_count_list = []
  words_dict= OrderedDict()
  for word in tokens:
    words_dict[word] = words_dict.get(word,0) + 1
  for item in words_dict.items():
    word_count_list.append((item[0],(id,item[1])))
  return word_count_list

def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples 
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  return  sorted(unsorted_pl,key=lambda y: y[0])

def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''

  return postings.mapValues(lambda x : len(x))

def partition_postings_and_write(postings,new_index_name):
  ''' A function that partitions the posting lists into buckets, writes out 
  all posting lists in a bucket to disk, and returns the posting locations for 
  each bucket. Partitioning should be done through the use of `token2bucket` 
  above. Writing to disk should use the function  `write_a_posting_list`, a 
  static method implemented in inverted_index_colab.py under the InvertedIndex 
  class. 
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and 
      offsets its posting list was written to. See `write_a_posting_list` for 
      more details.
  '''
  #partitioning RDD to (bucket_id, [(w0, posting_list_0)) entries 
  postings = postings.map(lambda x: (new_index_name+str(token2bucket_id(x[0])),x))
  #reducing it to (bucket_id, [(w0, posting_list_0), (w1, posting_list_1), ...])
  postings = postings.groupByKey()
  #writing each bucket to disk
  #need to change it in gcp to:
  postings = postings.map(lambda y: (InvertedIndex.write_a_posting_list(y,'3b1u8c3k6e4t122')))
  return postings
  
def calc_bm25(doc_len, avgdl,k,b):
  
  return k * (1 - b + b * (doc_len/avgdl))

In [ ]:
def create_body_index_with_bm_25_weights(data,new_index_name):
    # word counts map - posting lists
    word_counts = data.flatMap(lambda x: word_count(x[1], x[0]))
    #united posting lists
    postings = word_counts.groupByKey().mapValues(reduce_word_counts)
    #Next, we will filter out rare words, words that appear in 10 or fewer documents (when working on the entire corpus, we will increase this threshold to a minimum of 50 documents).
    #in gcp: change filter to 50
    postings = postings.filter(lambda x: len(x[1])>50) 
    # RDD to map {doc_id,[(token,tf),(token,tf)....]}
    tf = postings.flatMap(lambda x: [(tup[0], (x[0],tup[1])) for tup in x[1]] ) 
    # groupBy doc_id
    tf = tf.groupByKey().mapValues(list)
    #RDD to map doc_id to doc_len
    doc_lens_rdd = tf.map(lambda x: (x[0],builtins.sum([tup[1] for tup in x[1]])))
    # turn the doc_lens RDD into dict - TODO: check how its gonna be for the whole corpus
    doc_lens = doc_lens_rdd.collectAsMap()
    # filtering postings and calculate df
    w2df = calculate_df(postings)
    w2df_dict = w2df.collectAsMap()

    avgdl = builtins.sum(doc_lens.values())/len(doc_lens)
    #write posting lists to disk and returns posting locs dict for each bucket
    posting_locs_list = partition_postings_and_write(postings,new_index_name).collect()
    

    # collect all posting lists locations into one super-set- gcp code:
    super_posting_locs = defaultdict(list)
    for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
      if not blob.name.endswith("pickle"):
        continue
      elif not blob.name.startswith(f'postings_gcp/{new_index_name}'):
        continue
      with blob.open("rb") as f:
        posting_locs = pickle.load(f)
        for k, v in posting_locs.items():
          super_posting_locs[k].extend(v)

    # merge the posting locations into a single dict- good for local test in colab
#     super_posting_locs = defaultdict(list)
#     for posting_loc in posting_locs_list:
#         for k, v in posting_loc.items():
#           super_posting_locs[k].extend(v)

    new_index = InvertedIndex()
    # Adding the posting locations dictionary to the inverted index
    new_index.posting_locs = super_posting_locs
    # Add the token - df dictionary to the inverted index
    new_index.df = w2df_dict
    # the number of the docs in the corpus - using to calc TF-IDF
    new_index.N= data.count()
    #{doc_id:bm25_mechane}
    bm25 = doc_lens_rdd.map(lambda x: (x[0],calc_bm25(x[1], avgdl,1.5,0.75)))
    
    new_index.weights = bm25.collectAsMap()
    # write the global stats out
    new_index.write_index('.', new_index_name)

    return new_index

In [ ]:
def create_title_anchor_index_with_bm_25_weights(data,new_index_name):
    # word counts map - posting lists
    word_counts = data.flatMap(lambda x: word_count(x[1], x[0]))
    #united posting lists
    postings = word_counts.groupByKey().mapValues(reduce_word_counts)
    #Next, we will filter out rare words, words that appear in 10 or fewer documents (when working on the entire corpus, we will increase this threshold to a minimum of 50 documents).
    #in gcp: change filter to 50
    #postings = postings.filter(lambda x: len(x[1])>50) 
    # RDD to map {doc_id,[(token,tf),(token,tf)....]}
    tf = postings.flatMap(lambda x: [(tup[0], (x[0],tup[1])) for tup in x[1]] ) 
    # groupBy doc_id
    tf = tf.groupByKey().mapValues(list)
    #RDD to map doc_id to doc_len
    doc_lens_rdd = tf.map(lambda x: (x[0],builtins.sum([tup[1] for tup in x[1]])))
    # turn the doc_lens RDD into dict - TODO: check how its gonna be for the whole corpus
    doc_lens = doc_lens_rdd.collectAsMap()
    # filtering postings and calculate df
    w2df = calculate_df(postings)
    w2df_dict = w2df.collectAsMap()

    avgdl = builtins.sum(doc_lens.values())/len(doc_lens)
    #write posting lists to disk and returns posting locs dict for each bucket
    posting_locs_list = partition_postings_and_write(postings,new_index_name).collect()
    

    # collect all posting lists locations into one super-set- gcp code:
    super_posting_locs = defaultdict(list)
    for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
      if not blob.name.endswith("pickle"):
        continue
      elif not blob.name.startswith(f'postings_gcp/{new_index_name}'):
        continue
      with blob.open("rb") as f:
        posting_locs = pickle.load(f)
        for k, v in posting_locs.items():
          super_posting_locs[k].extend(v)

    # merge the posting locations into a single dict- good for local test in colab
#     super_posting_locs = defaultdict(list)
#     for posting_loc in posting_locs_list:
#         for k, v in posting_loc.items():
#           super_posting_locs[k].extend(v)

    new_index = InvertedIndex()
    # Adding the posting locations dictionary to the inverted index
    new_index.posting_locs = super_posting_locs
    # Add the token - df dictionary to the inverted index
    new_index.df = w2df_dict
    # the number of the docs in the corpus - using to calc TF-IDF
    new_index.N= data.count()
    #{doc_id:bm25_mechane}
    bm25 = doc_lens_rdd.map(lambda x: (x[0],calc_bm25(x[1], avgdl,1.5,0.75)))
    
    new_index.weights = bm25.collectAsMap()
    # write the global stats out
    new_index.write_index('.', new_index_name)

    return new_index

In [ ]:
bm25_B= create_body_index_with_bm_25_weights(body_data,"bm25_B")
print("done creating body")

In [ ]:
index_src = "bm25_B.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp/{index_src}'

In [ ]:
bm25_T= create_title_anchor_index_with_bm_25_weights(title_data,"bm25_T")
print("done creating title")

In [ ]:
index_src = "bm25_T.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp/{index_src}'

In [ ]:
bm25_A= create_title_anchor_index_with_bm_25_weights(anchor_data,"bm25_A")
print("done creating anchor")

In [ ]:
index_src = "bm25_A.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp/{index_src}'